# $\color{Green}{\text{Fundamentals of Inference and Learning}}$ 
Name: Roduit \
Surname : Vincent \
Sciper: 325140

Again, as said in Homework 1, only relevant steps of the calculation are written in this notebook. The full derivation can be found in the PDF file attached to the submission
## Homework 3
### Exercise 1 : Backpropagation with logistic loss

In [27]:
import numpy as np
import torch

In [28]:
import warnings
warnings.filterwarnings('ignore')

1.

In [29]:
def sigmoid(t):
    """Compute the sigmoid function.
    Args:
        t (np.ndarray): Input array of any shape
    Returns:
        np.ndarray, sigmoid function applied to input array
    """
    return 1 / (1 + np.exp(-t))

In [30]:
def predict(X,W):
    """ Compute the prediction
    Args :
        X (np.array): input data of shape (B,D)
        Dictionary W (dict): dictionary of weights (W={w 1: D x K, w 2: K x 1})
    Returns:
        tuple of np.ndarray, z_1, z_2, y_hat
    """
    z_1 = np.dot(X,W['w1'])
    x_1 = sigmoid(z_1)
    z_2 = np.dot(x_1,W['w2'])
    y_hat = sigmoid(z_2)
    return (z_1,z_2,y_hat)

2. 

In [31]:
def logistic_loss(y, y_hat):
    """ Compute the logistic loss function
    Args :
        y (np.array): input data of shape (B,1)
        y_hat (np.array): input data of shape (B,1)
    Output :
        loss (float): logistic loss
    """
    return -np.mean(y*np.log(y_hat)+(1-y)*np.log(1-y_hat))

In [32]:
B = 10
y = np.zeros(B)
y_hat = np.zeros(B)
loss = logistic_loss(y, y_hat)
print(f'The loss is  L={loss:.4f}')

The loss is  L=nan


3. 

In [33]:
def stable_logistic_loss(y, z_2):
    """ Compute the logistic loss function
    Args :
        y (np.array): input data of shape (B,1)
        z_2 (np.array): input data of shape (B,1)
    Output :
        loss (float): logistic loss
    """
    return np.mean(np.logaddexp(0,-z_2) + (1-y)*z_2)

In [34]:
z_2 = -10e10 * np.ones(B)
y = np.zeros(B)

loss = stable_logistic_loss(y, z_2)
print(f'The loss is: L={loss:.2f}')

The loss is: L=0.00


4. 

5.

In [ ]:
def gradient(X,y,W):
    """ Compute the gradient of the logistic loss function
    Args :
        X (np.array): input data of shape (B,D)
        y (np.array): input data of shape (B,1)
        W (dict): dictionary of weights (W={w 1: D x K, w 2: K x 1})
    Output :
        grad (dict): dictionary of gradients (grad={w 1: D x K, w 2: K x 1})
    """